In [5]:
import sys
import site
site.addsitedir('D:/documents/data/onderzoek/boekensites/corpus odbr/programs/analyse')
print(sys.path)
import pandas as pd
import numpy as np
import psycopg2
import taggedString as ts
import re
from collections import Counter
%load_ext autoreload
%autoreload 2
conn = psycopg2.connect("dbname=odbr user=odbrread")
curs = conn.cursor()
tsqh = ts.tsQueryHelper('word')

['', 'D:\\Python\\WinPython-64bit-3.6.1.0Qt5\\python-3.6.1.amd64\\python36.zip', 'D:\\Python\\WinPython-64bit-3.6.1.0Qt5\\python-3.6.1.amd64\\DLLs', 'D:\\Python\\WinPython-64bit-3.6.1.0Qt5\\python-3.6.1.amd64\\lib', 'D:\\Python\\WinPython-64bit-3.6.1.0Qt5\\python-3.6.1.amd64', 'D:\\Python\\WinPython-64bit-3.6.1.0Qt5\\python-3.6.1.amd64\\lib\\site-packages', 'd:\\documents\\data\\onderzoek\\liwctools', 'D:\\Python\\WinPython-64bit-3.6.1.0Qt5\\python-3.6.1.amd64\\lib\\site-packages\\win32', 'D:\\Python\\WinPython-64bit-3.6.1.0Qt5\\python-3.6.1.amd64\\lib\\site-packages\\win32\\lib', 'D:\\Python\\WinPython-64bit-3.6.1.0Qt5\\python-3.6.1.amd64\\lib\\site-packages\\Pythonwin', 'D:\\Python\\WinPython-64bit-3.6.1.0Qt5\\python-3.6.1.amd64\\lib\\site-packages\\IPython\\extensions', 'D:\\Python\\WinPython-64bit-3.6.1.0Qt5\\settings\\.ipython', 'D:\\documents\\data\\onderzoek\\boekensites\\corpus odbr\\programs\\analyse']
The autoreload extension is already loaded. To reload it, use:
  %reload_ex

In [6]:
import shelve
codeShelve = shelve.open('D:/documents/data/onderzoek/boekensites/IGEL 2018 views on literature/repo/data/codeshelve')

In [7]:
keys = [key for key in codeShelve.keys()]
nkeys = len(keys)
ncodes = sum(len(codeShelve[key]) for key in keys)
print(nkeys, ncodes)

96 409


In [8]:
import string
aspects = [char for char in string.ascii_uppercase[:12]]
characteristics = range(1,21)
ends = ['1!','1','0','-1','-1!']
polarity = ['p','u','n']
perspectives = ['r','a','w','g','t','f']

In [9]:
curs.execute("""
select bor.responseid, rte.text 
            from bookresponse bor, book_bookresponse_rel bbr, book boo, responsetext rte
            where bor.responseid = bbr.responseid
            and bor.responseid = rte.responseid
            and articlegenre = 'review'
            and author like '%Wieringa%'
            and author like '%Tommy%'
            and bbr.bookid = boo.bookid
            and texttypeid = 2
            order by bor.date, bor.time
            limit 10000""")

In [10]:
from taggedString import meta
sents = {}
for c in curs:
    tsc = ts.taggedStringCreator()
    s = tsc.create(c[0],c[1],meta(None,None,None,None))
    i = 1
    ident = str(c[0]) + '.' + str(i)
    sent = ts.taggedString(ident,'',meta(None,None,None,None))
    for tw in s:
        sent.append(tw)
        if tw.lemma in ['.','?','!']:
            sent.repos()
            sents[ident] = sent
            i += 1
            ident = str(c[0]) + '.' + str(i)
            sent = ts.taggedString(ident,'',meta(None,None,None,None))
    if len(sent) > 0:
        sent.repos()
        sents[ident] = sent


In [11]:
print(len(sents.keys()))

2872


In [12]:
rules = pd.read_csv('D:/documents/data/onderzoek/boekensites/IGEL 2018 views on literature/repo/data/wieringa.txt',
                       sep='\t',header=None,names=['cql','code'],quotechar="'")
len(rules)

314

In [13]:
rules.iloc[0,]['cql']

'"leest""lekker"'

In [14]:
queries = []
for i in range(len(rules)):
    queries.append(tsqh.translate(rules.iloc[i,]['cql']))
print(queries[0])

"leest""lekker"
a " 0
a " 7
"verhaallijn.*"
a " 0
"vlot""geschreven"
a " 0
a " 6
"ontknoping"
a " 0
"een""genot""om"
a " 0
a " 5
a " 12
"in"[]{0,3}[lemma="uitlezen"]
a " 0
b [ 4
d { 6
0 3
e } 10
b [ 11
a " 18
c ] 28
[lemma="mooi"][word="taal.*"]
b [ 0
a " 7
c ] 13
b [ 14
a " 20
c ] 28
"vaderfiguur"
a " 0
"ontspannend"
a " 0
[lemma="ongeloofwaardig"]
b [ 0
a " 7
c ] 24
"weinig"[]*[lemma="diepgang"]
a " 0
b [ 8
d * 10
0 9999
b [ 11
a " 18
c ] 28
"geen|zonder"[]{0,3}[lemma="diepgang"]
a " 0
b [ 13
d { 15
0 3
e } 19
b [ 20
a " 27
c ] 37
"lekker|goed""leesbaar"
a " 0
a " 13
"meeslep.*"[]?"verhaa?l.*"
a " 0
b [ 11
d ? 13
0 1
a " 14
"meeslep.*"[]?"boek.*"
a " 0
b [ 11
d ? 13
0 1
a " 14
"schrijfstijl"
a " 0
"ga""je"[]?"op"
a " 0
a " 4
b [ 8
d ? 10
0 1
a " 11
"schitterend.*""boek.*"
a " 0
a " 15
"schitterend.*"
a " 0
"hoofdperso.*"
a " 0
"bijzonder.*""verhaa?l.*"
a " 0
a " 13
"diepzinnig"
a " 0
"echte""personage.*"
a " 0
a " 7
"verhaal"[]{0,4}"lagen|gelaagd.*"
a " 0
b [ 9
d { 11
0 4
e } 15
a " 

b [ 60
d { 62
0 3
e } 66
a " 67
"personage.?|karakter.?"[word!="geen|niet|onvoldoende|amper|minimaal|half|nauwelijks"]{0,4}"uitgewerkt"
a " 0
b [ 24
a " 31
c ] 85
d { 86
0 4
e } 90
a " 91
<s>[word!="geen|niet|amper|nauwelijks"]*"lelijke""zin.*"
b [ 3
a " 10
c ] 38
d * 39
0 9999
a " 40
a " 49
"positie.*"[]{0,3}"gehoord"
a " 0
b [ 11
d { 13
0 3
e } 17
a " 18
"persoo?n.*|karakter.?"[word!="niet"]{0,5}"kleurrijk"
a " 0
b [ 23
a " 30
c ] 36
d { 37
0 5
e } 41
a " 42
"stijl|schrijfstijl"[word!="niet"]{0,5}"mooi"
a " 0
b [ 20
a " 27
c ] 33
d { 34
0 5
e } 38
a " 39
"kort""en""bondig"
a " 0
a " 6
a " 10
"drama|dramatisch"
a " 0
"feelgood"
a " 0
"mooi|knap|geweldig|echt|goed"[]{0,2}"neergezet"
a " 0
b [ 30
d { 32
0 2
e } 36
a " 37
"titel"[word!="niet|weinig"]{0,3}"aanspr.*"
a " 0
b [ 7
a " 14
c ] 27
d { 28
0 3
e } 32
a " 33
"geen""lange""zinnen"
a " 0
a " 6
a " 13
"geen"[]{0,3}"moeilijk.?""woorden|taal.*|zin.*|stijl|schrijfstijl"
a " 0
b [ 6
d { 8
0 3
e } 12
a " 13
a " 25
"geen"[]{0,3}"moeilijk.?

In [15]:
compcodes = {}
for key in sents.keys():
    sent = sents[key]
    ident = sent.id.partition('.')[0]
    for i in range(len(rules)):
        code = rules.iloc[i,]['code']
        if tsqh.tsqueryexec(queries[i],sents[key]):
            if ident in compcodes:
                if code not in compcodes[ident]:
                    compcodes[ident].append(code)
            else:
                compcodes[ident] = [code]
compcodes

{'1037611': ['K 18 1 p', 'A 15'],
 '1049303': ['D 15 1 p', 'D 18', 'H 15', 'A 15 1 p', 'A 15', 'K 18'],
 '1051177': ['F 15', 'F 21', 'K 6 1 p'],
 '1051199': ['D 15 1 p', 'K 6 1 p', 'F 15', 'C 15'],
 '1051208': ['H 15'],
 '1051214': ['K 18 1 p',
  'K 9 1 p',
  'D 4 -1',
  'A 15 1 p',
  'C 15',
  'F 15',
  'F 4 1 p',
  'A 1 1 p',
  'H 15',
  'K 6 1 p',
  'K 6',
  'K 9',
  'K 16 1 p'],
 '1051218': ['K 15 1 p f', 'F 15'],
 '1051225': ['K 15 1 p', 'F 15', 'K 4 1 p'],
 '1051226': ['K 15 1 p f', 'F 15'],
 '1051230': ['K 6 1 p', 'H 15', 'A 15 1 p', 'K 5-9'],
 '1051244': ['A 4 1 p', 'A 15 n', 'K 8 -1 n', 'F 15 0 u'],
 '1051279': ['H 15', 'H 8-9 1 p', 'K 18 -1 n', 'F 15'],
 '1051311': ['H 15', 'H 8-9 1 p', 'K 18 -1 n', 'F 15'],
 '1054109': ['K 18 1 p'],
 '1054118': ['A 15', 'F 15'],
 '105876': ['F 15', 'A 15 0 p'],
 '105914': ['A 15 0 p'],
 '1062106': ['K 5 1 p', 'A 15', 'A 15 1 p'],
 '106493': ['A 15 1 p', 'K 5-9', 'D 15', 'K 9 1 p'],
 '107057': ['A 15 0 p', 'A 15 1 p', 'K 5-9'],
 '139529': ['C

In [16]:
keys = [key for key in compcodes.keys()]
nkeys = len(keys)
ncodes = sum(len(compcodes[key]) for key in keys)
print(nkeys, ncodes)

335 977


In [17]:
def shortlabel(l):
    if l == 'asp2':
        return 'asp'
    elif l == 'char2':
        return 'char'
    else:
        return l
def outcomes(codeset):
    outcome = {}
    patt = '(?P<asp>[A-L])(-(?P<asp2>[A-L]))?\s(?P<char>[1-9]|1[0-9]|20|21)(-(?P<char2>[1-9]|(1[0-9])|20|21))?(\s(?P<end>1!|1|0|-1|-1!))?(\s(?P<pol>p|u|n))?(\s(?P<persp>r|a|g|w|t|f))?$'
    for key in codeset.keys():
        outcome[key] = Counter()
        for code in codeset[key]:
            mo = re.match(patt, code)        
            for f in ['asp','asp2','char','char2','end','pol','persp']:
                if mo.group(f):
                    outcome[key][shortlabel(f) + '-' + str(mo.group(f))] += 1
            linders = mo.group('asp')
            if mo.group('asp2'): linders += '-' + mo.group('asp2')
            linders += ' ' + mo.group('char')
            if mo.group('char2'): linders += '-' + mo.group('char2')
            outcome[key][linders] += 1
            if 'char-None' in outcome[key]:
                print('betrapt')
                print(key,codeset[key],outcome[key])
    df = pd.DataFrame(outcome)
    df.fillna(0,inplace=True)
    return df.transpose()


In [18]:
dfp = outcomes(compcodes)
dfc = outcomes(codeShelve)
print(dfc.columns)

Index(['A 1', 'A 14', 'A 15', 'A 17', 'A 18', 'A 20', 'A 21', 'A 4', 'A 5',
       'A 6',
       ...
       'end-1', 'end-1!', 'persp-f', 'persp-g', 'persp-r', 'persp-t',
       'persp-w', 'pol-n', 'pol-p', 'pol-u'],
      dtype='object', length=143)


In [19]:
rows = np.intersect1d(dfp.index, dfc.index)
cols = np.intersect1d(dfp.columns, dfc.columns)

In [20]:
dfpl = dfp.loc[rows,cols]
dfcl = dfc.loc[rows,cols]
dfcl

,A 1,A 14,A 15,A 17,A 18,A 20,A 21,A 4,A 5,A 6,...,end--1,end-0,end-1,persp-f,persp-r,persp-t,persp-w,pol-n,pol-p,pol-u
1051214,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0
1051218,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
139537,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0
139539,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
148080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
148082,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
148084,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
199166,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,3.0,0.0
199939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
200383,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0


In [21]:
#from scipy.stats.stats import pearsonr,spearmanr
#cors = {col: spearmanr(dfpl[col],dfcl[col]) for col in cols}

for cor in sorted(cors.keys()):
    print(cor, cors[cor])
lkeys = set(cors.keys())
for l in lkeys.copy():
    if l[0:2] in ('as','ch','en','pe','po'):
        lkeys.remove(l)
print(np.mean([cors[cor][0] for cor in lkeys]))
for cor in sorted(lkeys):
    print(cor, cors[cor][0])


In [22]:
def check(code,dfpl, dfcl):
#    keys1 = sorted([key for key in codedict1.keys() if matches(code, codedict1[key])])
#    keys2 = sorted([key for key in codedict2.keys() if matches(code, codedict2[key])])
#    print(code)
    setp = set(dfpl[dfpl[code] > 0].index)
    setc = set(dfcl[dfcl[code] > 0].index)
#    print('In p: ',sorted(setp - setc))
#    print('Shared: ', sorted(setp & setc))
#   print('In c: ',sorted(setc - setp))
    recall = len(setp & setc) / len(setc)
    prec = (len(setp & setc) / len(setp)) if setp else 1
    f = (2 * recall * prec / (recall + prec)) if (recall + prec ) > 0 else 0
    return {"code":code, "coded": len(setc), "recall": recall, "precision": prec, "f": f }

In [23]:
setp = set(dfpl[dfpl['A 1'] > 0].index)
setp

{'1051214'}

In [24]:
check('A 17',dfpl,dfcl)

{'code': 'A 17', 'coded': 1, 'f': 1.0, 'precision': 1.0, 'recall': 1.0}

In [25]:
output = [check(col,dfpl,dfcl) for col in sorted(cols)]
output = pd.DataFrame(output)
output
    

,code,coded,f,precision,recall
0,A 1,2,0.666667,1.000000,0.500000
1,A 14,6,0.800000,1.000000,0.666667
2,A 15,24,0.736842,0.636364,0.875000
3,A 17,1,1.000000,1.000000,1.000000
4,A 18,1,0.000000,0.000000,0.000000
5,A 20,3,0.800000,1.000000,0.666667
6,A 21,1,1.000000,1.000000,1.000000
7,A 4,6,0.727273,0.800000,0.666667
8,A 5,1,0.000000,0.000000,0.000000
9,A 6,3,0.800000,1.000000,0.666667


In [26]:
output.loc[[True if c.startswith("asp") else False for c in list(output["code"])]]

,code,coded,f,precision,recall
63,asp-A,46,0.886364,0.928571,0.847826
64,asp-B,3,1.000000,1.000000,1.000000
65,asp-C,19,0.538462,1.000000,0.368421
66,asp-D,41,0.805556,0.935484,0.707317
67,asp-F,32,0.692308,0.900000,0.562500
68,asp-G,12,0.500000,1.000000,0.333333
69,asp-H,8,0.800000,0.857143,0.750000
70,asp-I,4,0.857143,1.000000,0.750000
71,asp-J,24,0.631579,0.857143,0.500000
72,asp-K,86,0.965517,0.954545,0.976744


In [27]:
output.loc[[True if c.startswith("char") else False for c in list(output["code"])]]

,code,coded,f,precision,recall
73,char-1,9,0.705882,0.750000,0.666667
74,char-10,1,1.000000,1.000000,1.000000
75,char-12,1,1.000000,1.000000,1.000000
76,char-13,4,0.333333,0.500000,0.250000
77,char-14,15,0.896552,0.928571,0.866667
78,char-15,86,0.921212,0.962025,0.883721
79,char-16,4,0.857143,1.000000,0.750000
80,char-17,2,1.000000,1.000000,1.000000
81,char-18,32,0.787879,0.764706,0.812500
82,char-19,4,0.857143,1.000000,0.750000


In [28]:
output.loc[[True if c.startswith("pol") else False for c in list(output["code"])]]

,code,coded,f,precision,recall
100,pol-n,45,0.684932,0.892857,0.555556
101,pol-p,85,0.958084,0.975610,0.941176
102,pol-u,10,0.210526,0.222222,0.200000


In [29]:
output.loc[[True if c.startswith("per") else False for c in list(output["code"])]]

,code,coded,f,precision,recall
96,persp-f,10,0.666667,1.00,0.500000
97,persp-r,10,0.333333,1.00,0.200000
98,persp-t,6,0.285714,1.00,0.166667
99,persp-w,14,0.111111,0.25,0.071429


In [30]:
output.loc[[True if c.startswith("end") else False for c in list(output["code"])]]

,code,coded,f,precision,recall
93,end--1,45,0.779221,0.937500,0.666667
94,end-0,12,0.200000,0.250000,0.166667
95,end-1,81,0.932515,0.926829,0.938272


In [31]:
output.loc[[True if not (c.startswith("per") or c.startswith("end") or c.startswith("asp") or c.startswith("char") or c.startswith("pol")) else False for c in list(output["code"])]]

,code,coded,f,precision,recall
0,A 1,2,0.666667,1.000000,0.500000
1,A 14,6,0.800000,1.000000,0.666667
2,A 15,24,0.736842,0.636364,0.875000
3,A 17,1,1.000000,1.000000,1.000000
4,A 18,1,0.000000,0.000000,0.000000
5,A 20,3,0.800000,1.000000,0.666667
6,A 21,1,1.000000,1.000000,1.000000
7,A 4,6,0.727273,0.800000,0.666667
8,A 5,1,0.000000,0.000000,0.000000
9,A 6,3,0.800000,1.000000,0.666667


In [33]:
output.to_csv('D:/documents/data/onderzoek/boekensites/IGEL 2018 views on literature/repo/data/output.tsv',
                       sep='\t',header=True,index=False, quotechar="'",decimal=",")